# Work with MongoDB Atlas

A good Website with examples [https://www.simplifiedpython.net/](https://www.simplifiedpython.net/python-mongodb-connection/)

In [2]:
credentials = input('user:password')
connectionstring = "mongodb+srv://"+credentials+"@cluster0.epl3x.mongodb.net/?retryWrites=true&w=majority"

## SELECT \* FROM table

Connect to the remote Mongo DB and return all document

**<mark>PoweredOffVMs</mark>**

In [372]:
def get_poweredoffvms(connectionstring, database, collection):
    '''Connect to MongoDB and return data'''
    import pymongo, dns
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[database]
    mongo_col    = mongo_db[collection]
    mongo_client.close()
    return mongo_col.find()

result = get_poweredoffvms(connectionstring, 'JupyterNB', 'PoweredOffVMs')
for doc in result:
    print(doc)

{'_id': ObjectId('6055e8d45d839f102fa18107'), 'PSComputerName': 'vsc0000925.child.domain.com', 'PowerState': 'Off'}
{'_id': ObjectId('60560b905d839f102fa18114'), 'PSComputerName': 'TIN87500803.child.domain.com', 'PowerState': 'Off'}
{'_id': ObjectId('605628aa5d839f102fa18237'), 'PSComputerName': 'xsc0000983.child.domain.com', 'PowerState': 'Off'}
{'_id': ObjectId('60562f255d839f102fa18242'), 'PSComputerName': 'xsc0000989.child.domain.com', 'PowerState': 'Off'}


**<mark>FailedPatching</mark>**

In [371]:
def get_failedpatching(connectionstring, database, collection):
    '''Connect to MongoDB and return data'''
    import pymongo, dns
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[database]
    mongo_col    = mongo_db[collection]
    mongo_client.close()
    return mongo_col.find()

result = get_failedpatching(connectionstring, 'JupyterNB', 'FailedPatching')
for doc in result:
    print(doc)

{'_id': ObjectId('6055e92d5d839f102fa1810c'), 'RunspaceId': '0a6c50fe-7a2a-4225-9f27-85dad48d2b74', 'PSComputerName': 'TIN87500803.child.domain.com', 'LastPatchRun': '2021.03.03 05:00:33', 'LastPatchStatus': '', 'UpdateServer': 'http://srv2dp-1.prd.child.domain.com:8530', 'UpdateServerStatus': 'OK', 'CcmExecVersion': '5.00.8968.1000', 'CcmExecInstallDate': '20201216', 'CCMManagementPoint': 'srv2dp-2.prd.child.domain.com', 'CCMCimInstanceStatus': 'Success'}
{'_id': ObjectId('6055e92e5d839f102fa1810d'), 'RunspaceId': '0ea06798-482a-4628-869b-a190bf1df050', 'PSComputerName': 'vsc0000925.child.domain.com', 'LastPatchRun': '', 'LastPatchStatus': '', 'UpdateServer': 'http://srv2dp-1.prd.child.domain.com:8530', 'UpdateServerStatus': 'OK', 'CcmExecVersion': '5.00.8968.1000', 'CcmExecInstallDate': '20201103', 'CCMManagementPoint': 'srv2dp-1.prd.child.domain.com', 'CCMCimInstanceStatus': 'Success'}
{'_id': ObjectId('6055e92e5d839f102fa1810e'), 'RunspaceId': 'abf4fdaf-e1fe-45bf-8977-e5b1f01b4519'

**<mark>JOIN FailedPatching and PoweredOffVMs</mark>**

In [370]:
def get_data(connectionstring, mongodatabase, collection):
    '''Connect to MongoDB and return data'''
    import pymongo, dns
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[mongodatabase]
    mongo_col    = mongo_db[collection]
    result       = mongo_col.find().sort("PSComputerName")
    mongo_client.close()
    return result

import pandas as pd
failed_and_poweroff = []
for failedpatching in get_data(connectionstring, 'JupyterNB', 'FailedPatching'):
    for poweredoffvms in get_data(connectionstring, 'JupyterNB', 'PoweredOffVMs'):
        if failedpatching['PSComputerName'] == poweredoffvms['PSComputerName']:
            thisdict = {                                
                    #'RunspaceId'           : failedpatching['RunspaceId'],
                    'PSComputerName'       : failedpatching['PSComputerName'],
                    'LastPatchRun'         : failedpatching['LastPatchRun'],
                    'LastPatchStatus'      : failedpatching['LastPatchStatus'],
                    'UpdateServerStatus'   : failedpatching['UpdateServerStatus'],
                    'CcmExecVersion'       : failedpatching['CcmExecVersion'],
                    'CCMCimInstanceStatus' : failedpatching['CCMCimInstanceStatus'],
                    'PowerState'           : poweredoffvms['PowerState'],
            }
            failed_and_poweroff.append(thisdict)

# Print data frame set as table
df = pd.DataFrame(failed_and_poweroff)
print(df.to_string())


                 PSComputerName         LastPatchRun LastPatchStatus UpdateServerStatus  CcmExecVersion CCMCimInstanceStatus PowerState
0  TIN87500803.child.domain.com  2021.03.03 05:00:33                                 OK  5.00.8968.1000              Success        Off
1   vsc0000925.child.domain.com                                                      OK  5.00.8968.1000              Success        Off
2   xsc0000983.child.domain.com                                                      OK  5.00.8968.1000              Success        Off


## INSERT INTO table

Connect to the remote Mongo DB and insert one document

In [ ]:
def add_document(connectionstring, mongodatabase, collection, document):
    '''Connect to MongoDB and add a document'''
    import pymongo, dns
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[mongodatabase]
    mongo_col    = mongo_db[collection]
    mongo_col.insert_one(document)
    for doc in mongo_col.find():
        print(doc)

    mongo_client.close()

add_document(connectionstring, 'JupyterNB', 'PoweredOffVMs', { "PSComputerName": "xsc0000989.child.domain.com", "PowerState": "Off" })

## DELETE FROM table

Connect to the remote MongoDB and delete all Documents in the Collection PoweredOffVMs

In [ ]:
def delete_collection(connectionstring, mongodatabase, collection):
    '''Connect to MongoDB and add a document'''import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[mongodatabase]
    mongo_col    = mongo_db[collection]
    mongo_col.delete_many({})
    for doc in mongo_col.find():
        print(doc)

    mongo_client.close()

add_document(connectionstring, 'JupyterNB', 'PoweredOffVMs')

## Read from a CSV-file and print it out

In [ ]:
def import_csv(file):
    '''Import-CSV'''
    import csv, sys, os

    check_path = os.path.exists(file)
    if(check_path == True):

        with open(file, mode='r', newline='') as csv_file:
            csv_reader = csv.DictReader(csv_file, delimiter=';')
            try:
                for row in csv_reader:
                    print(row)

            except csv.Error as e:
                sys.exit('file {}, line {}: {}'.format(file, csv_reader.line_num, e))

    else:
        print(f'File not found: {file}')

file_to_import = "C:\\Users\\Tinu\\Downloads\\poweredOffVMs.csv"
import_csv(file_to_import)

## Read from a JSON-file and print it out

In [ ]:
# region define colors
class colors: 
    '''Colors class:reset all colors with colors.reset'''

    reset         = '\033[0m'
    bold          = '\033[01m'
    disable       = '\033[02m'
    underline     = '\033[04m'
    reverse       = '\033[07m'
    strikethrough = '\033[09m'
    invisible     = '\033[08m'

    class fg: 
        ''' Define the foreground-colors
        use as colors.fg.colorname
        '''
        black      = '\033[30m'
        red        = '\033[31m'
        green      = '\033[32m'
        orange     = '\033[33m'
        blue       = '\033[34m'
        purple     = '\033[35m'
        cyan       = '\033[36m'
        lightgrey  = '\033[37m'
        darkgrey   = '\033[90m'
        lightred   = '\033[91m'
        lightgreen = '\033[92m'
        yellow     = '\033[93m'
        lightblue  = '\033[94m'
        pink       = '\033[95m'
        lightcyan  = '\033[96m'

    class bg: 
        ''' Define the background-colors
        use as colors.bg.colorname
        '''
        black     = '\033[40m'
        red       = '\033[41m'
        green     = '\033[42m'
        orange    = '\033[43m'
        blue      = '\033[44m'
        purple    = '\033[45m'
        cyan      = '\033[46m'
        lightgrey = '\033[47m'
#endregion

In [ ]:
# Function to search a specified key and print out the defined rows from a JSON-file
def search_infile(file,search_identifier,search_field):
    '''You can use json.load() method to read a file containing JSON object.'''
    import os, json, sys
    with open(file, "r") as read_file:
        data = json.load(read_file)

    try:
        # Fetch through the list (array)
        for key, value in data.items():
            # Search for identifier
            if key == search_identifier:
                print(colors.fg.green + 'Found: ' + key + colors.reset)
                # Value as list object
                for row in value:
                    print(f'Computer: {row[search_field[0]]}\t{search_field[1]}: {row[search_field[1]]}')
                    #print(row)

    except:
        sys.exit('file {}'.format(file))

# Get defined results from the JSON-file
import os
file_to_import = "C:/Users/Admin/Downloads/PatchingAnalyse.json"
check_path = os.path.exists(file_to_import)
if(check_path == True):
    search_key    = 'WindowsUpdate'
    search_fields = ['PSComputerName','Connection']
    print(colors.fg.cyan + f'Search for Key "WindowsUpdate"' + colors.reset)
    search_infile(file_to_import, search_key, search_fields)

    search_key    = 'LastPatchRun'
    search_fields = ['PSComputerName','LastPatchRun']
    print(colors.fg.cyan + f'Search for Key "LastPatchRun"' + colors.reset)
    search_infile(file_to_import, search_key, search_fields)

    search_key    = 'LastPatchStatus'
    search_fields = ['PSComputerName','LastPatchStatus']
    print(colors.fg.cyan + f'Search for Key "LastPatchStatus"' + colors.reset)
    search_infile(file_to_import, search_key, search_fields)

    search_key    = 'SoftwareKiosk'
    search_fields = ['PSComputerName','State']
    print(colors.fg.cyan + f'Search for Key "SoftwareKiosk"' + colors.reset)
    search_infile(file_to_import, search_key, search_fields)

    search_key    = 'CumulativeUpdate'
    search_fields = ['PSComputerName','Description']
    print(colors.fg.cyan + f'Search for Key "CumulativeUpdate"' + colors.reset)
    search_infile(file_to_import, search_key, search_fields)

    search_key    = 'SCCMErrors'
    search_fields = ['PSComputerName','Message']
    print(colors.fg.cyan + f'Search for Key "SCCMErrors"' + colors.reset)
    search_infile(file_to_import, search_key, search_fields)

In [ ]:
# Function to read and print out the whole JSON-file
def list_jsonfile(file):
    '''You can use json.load() method to read a file containing JSON object.'''
    import os, json, sys
    with open(file, "r") as read_file:
        data = json.load(read_file)

    try:
        # Fetch through the list (array)
        for key, value in data.items():
            print(colors.fg.green + 'Found: ' + key + colors.reset)
            # Value as list object
            if value is None:
                continue
            else:
                for row in value:
                    print(row['RunspaceId'])

    except:
        sys.exit('file {}'.format(file))


# Get all results from the JSON-file
import os
file_to_import = "C:/Users/Admin/Downloads/PatchingAnalyse.json"
check_path = os.path.exists(file_to_import)
if(check_path == True):
    list_jsonfile(file_to_import)

In [ ]:
# Function to search for the specified key and print out the elements of it from a JSON-file
def search_infile(file,search_identifier):
    '''You can use json.load() method to read a file containing JSON object.'''
    import os, json, sys
    with open(file, "r") as read_file:
        data = json.load(read_file)

    try:
        # Fetch through the list (array)
        for item in search_identifier:
            print(colors.fg.green + 'Found: ' + item + colors.reset)
            for key, value in data.items():
                #print(colors.fg.green + 'Found: ' + key + colors.reset)
                if key == item:
                    # Value as list object
                    if value is None:
                        continue
                    else:
                        for row in value:
                            print(row)

    except:
        sys.exit('file {}'.format(file))

# Get all results from the JSON-file
import os
file_to_import = "C:/Users/Admin/Downloads/PatchingAnalyse.json"
check_path = os.path.exists(file_to_import)
if(check_path == True):
    search_identifier = ['LastPatchRun','LastPatchStatus','WindowsUpdate','SCCMAgent']
    search_infile(file_to_import,search_identifier)

## Read from a JSON-file and import into MongoDB Atlas

Connect to the remote Mongo DB and import data from JSON-file

In [ ]:
# Function to search for the specified key and return the elements of it from a JSON-file
def search_infile(file,search_identifier):
    '''You can use json.load() method to read a file containing JSON object.'''
    import json, sys

    with open(file, "r") as read_file:
        data = json.load(read_file)

    try:
        # Fetch through the list (array)
        for key, value in data.items():
            if key == search_identifier:
                if value is None:
                    continue
                else:
                    return_data = value
    except:
        sys.exit('file {}'.format(file))

    return return_data

# Get all results from the JSON-file
import os, pymongo, dns
mongo_client = pymongo.MongoClient(connectionstring)
mongo_db     = mongo_client['JupyterNB']
mongo_col    = mongo_db['FailedPatching']

file_to_import = "C:/Users/Admin/Downloads/PatchingAnalyse.json"
check_path     = os.path.exists(file_to_import)

if(check_path == True):
    
    # Create a new data and import into mongodb
    for patchrun in search_infile(file_to_import,'LastPatchRun'):
        for patchstatus in search_infile(file_to_import,'LastPatchStatus'):
            # Join LastPatchRun with LastPatchStatus
            if patchstatus['RunspaceId'] == patchrun['RunspaceId']:
                for winupd in search_infile(file_to_import,'WindowsUpdate'):
                    # Join LastPatchRun with LastPatchStatus and WindowsUpdate
                    if winupd['RunspaceId'] == patchrun['RunspaceId']:
                        for ccmexec in search_infile(file_to_import,'SCCMAgent'):
                            # Join LastPatchRun with LastPatchStatus, WindowsUpdate and SCCMAgent
                            if ccmexec['RunspaceId'] == patchrun['RunspaceId']:
                                for cimsccm in search_infile(file_to_import,'SCCMCimInstance'):
                                    # Join LastPatchRun with LastPatchStatus, WindowsUpdate, SCCMAgent and SCCMCimInstance
                                    if cimsccm['RunspaceId'] == patchrun['RunspaceId']:
                                        thisdict = {                                
                                            'RunspaceId'           : patchrun['RunspaceId'],
                                            'PSComputerName'       : patchrun['PSComputerName'],
                                            'LastPatchRun'         : patchrun['LastPatchRun'],
                                            'LastPatchStatus'      : patchstatus['LastPatchStatus'],
                                            'UpdateServer'         : winupd['WUServer'],
                                            'UpdateServerStatus'   : winupd['Connection'],
                                            'CcmExecVersion'       : ccmexec['Version'],
                                            'CcmExecInstallDate'   : ccmexec['InstallDate'],
                                            'CCMManagementPoint'   : cimsccm['ManagementPoint'],
                                            'CCMCimInstanceStatus' : cimsccm['Status']
                                        }
                                        mongo_col.insert_one(thisdict)

mongo_client.close()

## INSERT INTO table

Import-CSV in the local MongoDB Collection PoweredOffVMs

In [ ]:
def import_csv_mongodb(file, connectionstring, mongodatabase, collection):
    '''Import-CSV in the MongoDB Atlas Collection PoweredOffVMs'''
    import csv, sys, os, pymongo
    check_path = os.path.exists(file)
    if(check_path == True):
        
        mongo_client = pymongo.MongoClient(connectionstring)
        mongo_db     = mongo_client[mongodatabase]
        mongo_col    = mongo_db[collection]

        with open(file, mode='r', newline='') as csv_file:
            csv_reader = csv.DictReader(csv_file, delimiter=';')
            try:
                mongo_col.insert_many(csv_reader)

            except csv.Error as e:
                sys.exit('file {}, line {}: {}'.format(file, csv_reader.line_num, e))

        mongo_client.close()

    else:
        print(f'File not found: {file}')

file_to_import = "C:\\Users\\Tinu\\Downloads\\poweredOffVMs.csv"
import_csv_mongodb(file_to_import, connectionstring, 'JupyterNB', 'PoweredOffVMs')

## SELECT \* FROM table VALUES

## <span style="font-size: 14px;">Show tab-separated list</span>

In [ ]:
def print_collection(connectionstring, mongodatabase, collection):
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[mongodatabase]
    mongo_col    = mongo_db[collection]
    for doc in mongo_col.find():
        print('{0}\t{1}\t{2}\t{3}'.format(doc['PSComputerName'],doc['PowerState'],doc['LastPatched'],doc['PatchingWindow']))

    mongo_client.close()

print_collection(connectionstring, 'JupyterNB', 'PoweredOffVMs')

## SELECT \* FROM table WHERE VMName LIKE 'TINU005'

### <span style="font-size: 14px;">Connect to the remote Mongo DB and search for a document</span>

In [369]:
def search_for_document(connectionstring, mongodatabase, collection, filter):
    import pymongo, pandas as pd
    result = []
    mongo_client = pymongo.MongoClient(connectionstring))
    mongo_db     = mongo_client[mongodatabase]
    mongo_col    = mongo_db[collection]
    for doc in mongo_col.find(filter):
        #print('{0}\t{1}\t{2}\t{3}\t{4}\t{5}'.format(doc['LastRunTime'],doc['VMName'],doc['PSComputerName'],doc['PowerState'],doc['LastPatched'],doc['PatchingWindow']))
        thisdict = {                                
            #'RunspaceId'           : failedpatching['RunspaceId'],
            'PSComputerName'       : failedpatching['PSComputerName'],
            #'LastPatchRun'         : failedpatching['LastPatchRun'],
            'LastPatchStatus'      : failedpatching['LastPatchStatus'],
            'UpdateServerStatus'   : failedpatching['UpdateServerStatus'],
            'CcmExecVersion'       : failedpatching['CcmExecVersion'],
            'CCMCimInstanceStatus' : failedpatching['CCMCimInstanceStatus'],
        }
        result.append(thisdict)

    mongo_client.close()

    # Print data frame set as table
    df = pd.DataFrame(result)
    print(df.to_string())

searchdocument   = { "PSComputerName": "xsc0000585.child.domain.com" }
search_for_document(connectionstring, 'JupyterNB', 'FailedPatching', searchdocument)

                PSComputerName LastPatchStatus UpdateServerStatus  CcmExecVersion CCMCimInstanceStatus
0  xsc0000983.child.domain.com                                 OK  5.00.8968.1000              Success
